In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import numpy as np
import tensorflow as tf
path = r'C:\\Users\\weso\\Desktop\\glove.6B\\'
id_dict = {}
word_dict = {}

matrix = []
dimensions = 300
with open(path + "glove.6B.300d.txt", 'r' , encoding="utf-8")  as f:
    lines = f.readlines()
    vocab_size = len(lines)
    
    matrix = np.zeros((vocab_size,dimensions),dtype=float)
    for line in lines:
        values = line.split()
        word = values[0].strip()
        id = len(id_dict)
        id_dict[word]=id
        word_dict[id] = word
        vector = np.asarray(values[1:], "double")
        matrix[id_dict[word],:] = vector
print(len(id_dict))

400000


In [4]:
def find_candidate(sim,id_a,id_a_star,id_b):
    myList = [id_a, id_b, id_a_star];
    myList.sort();
    
    if(myList[0] != 0):
        index1 = np.argmax(sim[:myList[0]])
    else:
        index1 = None
    sim2 = sim[myList[0]+1:myList[1]]
    if len(sim2) > 0:
        index2 = np.argmax(sim2)
        index2 +=  myList[0]+1
    else:
        index2 = None
    sim3 = sim[myList[1]+1:myList[2]]
    if len(sim3) > 0:
        index3 = np.argmax(sim3)
        index3 +=  myList[1]+1
    else:
        index3 = None
    if(myList[2]+1 < len(sim)):
        index4 = np.argmax(sim[myList[2]+1:])
        index4 +=  myList[2]+1
    else:
        index4 = None
    myList = [index1, index2, index3,index4];
    values = []
    myList2 = []
    for index in myList:
        if index != None:
            values.append(sim[index])
            myList2.append(index)
    index = myList2[np.argmax(values)]
    return index

In [5]:
matrix_normalized = tf.nn.l2_normalize(matrix,axis = 1)

In [7]:
def find_nearest_k(searched_word,k):
    list = []
    id = id_dict[searched_word]
    searched_vector = matrix_normalized[id,:] 
    
    for word in id_dict:
        word_weights = matrix_normalized[id_dict[word]]
        loss = tf.tensordot(word_weights,searched_vector,axes = 1).numpy()
        list = insert(list,(word,loss))
        if len(list) > k:
            list = list[0:k+1]
    return list[0:k]

# Function to insert element
def insert(list, tuple):
    (word,n) = tuple
    if(len(list) == 0):
        list = [(word,n)]
    # Searching for the position
    for i in range(len(list)):
        (word_i,n_i) = list[i]
        if n_i < n:
            index = i
            break
      
    # Inserting n in the list
    list = list[:i] + [(word,n)] + list[i:]
    return list


print(find_nearest_k('batman',6))#this should produce one, for the same vectors
print(find_nearest_k('hogwarts',6))
print(find_nearest_k('turing',6))
print(find_nearest_k('florida',6))
print(find_nearest_k('object-oriented',6))
print(find_nearest_k('dancing',6))

[('batman', 1.0), ('superman', 0.6533917266554318), ('catwoman', 0.5784364578373318), ('superhero', 0.5565441009681275), ('spider-man', 0.5218831109165304), ('joker', 0.5129296574374288)]
[('hogwarts', 0.9999999999999998), ('dumbledore', 0.549229863608), ('wizardry', 0.5188968261681793), ('hermione', 0.4557605986849196), ('voldemort', 0.45157509151676123), ('hagrid', 0.4479209740521845)]
[('turing', 0.9999999999999998), ('deterministic', 0.5119534735350421), ('computable', 0.465302480354785), ('gödel', 0.45792113926794675), ('nondeterministic', 0.450346003250189), ('probabilistic', 0.43031487450612743)]
[('florida', 1.0000000000000002), ('miami', 0.6665895319646075), ('fla.', 0.6643667347542), ('carolina', 0.6428763922567252), ('texas', 0.6192212509245346), ('tallahassee', 0.5966705264371971)]
[('object-oriented', 0.9999999999999996), ('smalltalk', 0.6220564039940726), ('c++', 0.5849095890800992), ('scripting', 0.5446553503096182), ('implementations', 0.5332775991605072), ('domain-spec

## 3cosAdd 

In [8]:

def three_cos_predict_np(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix[id_a,:]
    weight_b = matrix[id_b,:]
    weight_a_star = matrix[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# 3cosAdd normalised befor arithmetic

In [9]:
def three_cos_predict_np_norm(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        return None
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    id_b_star = id_dict[b_star]#remove this after testing
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    direction = weight_b + ( weight_a_star - weight_a)
    direction /= np.linalg.norm(direction)
    
    sim = tf.tensordot(tf.convert_to_tensor(matrix_normalized),tf.convert_to_tensor(direction),axes = 1)
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

## 3cos Mult

In [10]:
def sim_01(matrix,weight):
    tmp = tf.tensordot(matrix,weight,axes=1)
    tmp = (tmp+1.0)/2.0
    return tmp

def predict_three_cos_mult(a,a_star,b,b_star):
    if( a in id_dict and b in id_dict and a_star in id_dict and b_star in id_dict):
        pass
    else:
        print(a in id_dict)
        print(a_star in id_dict)
        print(b in id_dict)
        print(b_star in id_dict)
        return None
    
    id_a = id_dict[a]
    id_b = id_dict[b]
    id_a_star = id_dict[a_star]
    
    weight_a = matrix_normalized[id_a,:]
    weight_b = matrix_normalized[id_b,:]
    weight_a_star = matrix_normalized[id_a_star,:]
    
    nominator = sim_01(matrix_normalized,weight_a_star) * sim_01(matrix_normalized,weight_b) 
    denominator = sim_01(matrix_normalized,weight_a) + 0.0001
    sim = nominator / denominator
    index = find_candidate(sim,id_a,id_a_star,id_b)
    return word_dict[index]

# Test all

In [11]:
count_sem_questions =  0
count_syn_questions =  0

count_sem_sucess      = [0,0,0]
count_syn_sucess      = [0,0,0]

errors = []


files = os.listdir('.\\datasets\\question-data')
for idx,file_name in enumerate(files):
    print(file_name)
    with open('.\\datasets\\question-data\\'+file_name,'r') as file:
        lines = file.readlines()
        local_sucesses    = [0,0,0]
        local_tasks_count = 0
        
        for line_id,line in enumerate(lines):
            if (line_id % 100 == 0):
                print(line_id)
            a,a_star,b,b_star = line.split(" ")
            b_star = b_star.strip()
            #predicted_3cos_add      = three_cos_predict_np(a,a_star,b,b_star)
            predicted_3cos_add_norm = three_cos_predict_np_norm(a,a_star,b,b_star)
            predicted_3cos_mult     = predict_three_cos_mult(a,a_star,b,b_star)
            #print(a,a_star,b,b_star)
            #print(predicted_3cos_add,predicted_3cos_add_norm,predicted_3cos_mult)
            if(idx < 5):#first 5 are sem
                count_sem_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_sem_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_sem_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_sem_sucess[2] += 1
            else:
                count_syn_questions += 1
                local_tasks_count   += 1
                
                #if predicted_3cos_add      == b_star:
                #    count_syn_sucess[0] += 1
                if predicted_3cos_add_norm == b_star:
                    local_sucesses[1] += 1
                    count_syn_sucess[1] += 1
                    
                if  predicted_3cos_mult    == b_star:
                    local_sucesses[2] += 1
                    count_syn_sucess[2] += 1
        print(local_sucesses[0]/float(local_tasks_count))
        print(local_sucesses[1]/float(local_tasks_count))
        print(local_sucesses[2]/float(local_tasks_count))
        print('-------------------')

print('semantical')

print(count_sem_sucess[0]/float(count_sem_questions))
print(count_sem_sucess[1]/float(count_sem_questions))
print(count_sem_sucess[2]/float(count_sem_questions))
        
print('syntactical')
print(count_syn_sucess[0]/float(count_sem_questions))
print(count_syn_sucess[1]/float(count_sem_questions))
print(count_syn_sucess[2]/float(count_sem_questions))

print('overall')
print((count_syn_sucess[0]+count_sem_sucess[0])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[1]+count_sem_sucess[1])/float(count_sem_questions+count_sem_questions))
print((count_syn_sucess[2]+count_sem_sucess[2])/float(count_sem_questions+count_sem_questions))

capital-common-countries.txt
0
100
200
300
400
500
0.0
0.9486166007905138
0.9545454545454546
-------------------
capital-world.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
0.0
0.9597701149425287
0.9566755083996463
-------------------
city-in-state.txt
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
0.0
0.5930279691933522
0.5755978921767329
-------------------
currency.txt
0
100
200
300
400
500
600
700
800
0.0
0.1581986143187067
0.19861431870669746
-------------------
family.txt
0
100
200
300
400
500
0.0
0.8814229249011858
0.8735177865612648
-------------------
gram1-adjective-to-adverb.txt
0
100
200
300
400
500
600
700
800
900
0.0
0.22580645161290322
0.2560483870967742
-------------------
gram2-opposite.txt
0
100
200
300
400
500
600
700
800
0.0
0.2

In [12]:
import winsound
winsound.Beep(440, 500)

Spearman Rank

In [13]:
with open(r'.\datasets\wordsim353_sim_rel\wordsim_similarity_goldstandard.txt') as file:

    losses = []
    scores = []

    lines = file.readlines()
    sum_diff = 0
    for line in lines:
        if(len(line) <= 1):
            pass

        print("line" + line)
        word1, word2, score10 = line.split()
        score10 = float(score10)
        try:
            print(word1,word2)
            id1 = id_dict[word1]
            id2 = id_dict[word2]
            #print(id1)
            #print(id2)
            vector1 = matrix_normalized[id1,:]
            vector2 = matrix_normalized[id2,:]
            #print(vector1)
            #print(vector2)
            loss = sim_01(vector1,vector2)
        except: 
            loss = 0
        print(str(loss),str(score10/10.0))
        sum_diff += abs((score10/10.0) - loss)
        losses.append(loss)
        scores.append(score10/10.0)
    print('Average Distance between prediction and hand assigned is ' + str(sum_diff / len(lines)))

linetiger	cat	7.35

tiger cat
tf.Tensor(0.6564453578418136, shape=(), dtype=float64) 0.735
linetiger	tiger	10.00

tiger tiger
tf.Tensor(1.0000000000000004, shape=(), dtype=float64) 1.0
lineplane	car	5.77

plane car
tf.Tensor(0.690760792411134, shape=(), dtype=float64) 0.577
linetrain	car	6.31

train car
tf.Tensor(0.7336681472205081, shape=(), dtype=float64) 0.631
linetelevision	radio	6.77

television radio
tf.Tensor(0.8394849865196272, shape=(), dtype=float64) 0.6769999999999999
linemedia	radio	7.42

media radio
tf.Tensor(0.7363814185458708, shape=(), dtype=float64) 0.742
linebread	butter	6.19

bread butter
tf.Tensor(0.8112817367483578, shape=(), dtype=float64) 0.619
linecucumber	potato	5.92

cucumber potato
tf.Tensor(0.7259734448685847, shape=(), dtype=float64) 0.592
linedoctor	nurse	7.00

doctor nurse
tf.Tensor(0.7929909078366437, shape=(), dtype=float64) 0.7
lineprofessor	doctor	6.62

professor doctor
tf.Tensor(0.7097683466835096, shape=(), dtype=float64) 0.662
linestudent	professor

linelobster	wine	5.70

lobster wine
tf.Tensor(0.6442794141628271, shape=(), dtype=float64) 0.5700000000000001
linechampionship	tournament	8.36

championship tournament
tf.Tensor(0.8588248390903788, shape=(), dtype=float64) 0.836
lineman	woman	8.30

man woman
tf.Tensor(0.849933168980951, shape=(), dtype=float64) 0.8300000000000001
lineman	governor	5.25

man governor
tf.Tensor(0.6265496930327097, shape=(), dtype=float64) 0.525
linemurder	manslaughter	8.53

murder manslaughter
tf.Tensor(0.7870661208656683, shape=(), dtype=float64) 0.853
lineopera	performance	6.88

opera performance
tf.Tensor(0.6950854450081975, shape=(), dtype=float64) 0.688
lineMexico	Brazil	7.44

Mexico Brazil
0 0.744
lineglass	metal	5.56

glass metal
tf.Tensor(0.7338108216544942, shape=(), dtype=float64) 0.5559999999999999
linealuminum	metal	7.83

aluminum metal
tf.Tensor(0.7741316211262262, shape=(), dtype=float64) 0.783
linerock	jazz	7.59

rock jazz
tf.Tensor(0.7454258965777212, shape=(), dtype=float64) 0.759
linemus

tf.Tensor(0.6387018162615964, shape=(), dtype=float64) 0.181
lineenergy	secretary	1.81

energy secretary
tf.Tensor(0.6304418193231902, shape=(), dtype=float64) 0.181
lineprecedent	group	1.77

precedent group
tf.Tensor(0.5086263477739716, shape=(), dtype=float64) 0.177
lineproduction	hike	1.75

production hike
tf.Tensor(0.5874976610037532, shape=(), dtype=float64) 0.175
linestock	phone	1.62

stock phone
tf.Tensor(0.6195062235373057, shape=(), dtype=float64) 0.162
lineholy	sex	1.62

holy sex
tf.Tensor(0.574958623407084, shape=(), dtype=float64) 0.162
linestock	CD	1.31

stock CD
0 0.131
linedrink	ear	1.31

drink ear
tf.Tensor(0.5512473930182669, shape=(), dtype=float64) 0.131
linedelay	racism	1.19

delay racism
tf.Tensor(0.543535842837384, shape=(), dtype=float64) 0.119
linestock	life	0.92

stock life
tf.Tensor(0.6135429640045588, shape=(), dtype=float64) 0.092
linestock	jaguar	0.92

stock jaguar
tf.Tensor(0.5316684850943864, shape=(), dtype=float64) 0.092
linemonk	slave	0.92

monk slave


In [14]:
from scipy import stats
stats.spearmanr(losses, scores)

SpearmanrResult(correlation=0.6386682409702303, pvalue=1.1678575503549501e-24)